In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy

from data import * 
from feature_engineering import FeaturesEngineering
from models import Models
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-02-23 19:22:23.317922: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 19:22:23.328291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740309743.340564   15823 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740309743.344407   15823 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 19:22:23.357324: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
pypickle_overwrite = True

In [3]:
dir_project = os.getcwd()
dir_models = os.path.join(dir_project, 'models')
dir_models_base = os.path.join(dir_project, 'models', 'base')
dir_models_scaler = os.path.join(dir_project, 'models', 'scaler')
dir_models_best = os.path.join(dir_project, 'models', 'best')

print(f"""
Project directory: {dir_project}
Models directory: {dir_models}
Base models directory: {dir_models_base}
Scalers directory: {dir_models_scaler}
Best models directory: {dir_models_best}
""")


Project directory: /home/alfred/projects/mscfe-masters-project/src
Models directory: /home/alfred/projects/mscfe-masters-project/src/models
Base models directory: /home/alfred/projects/mscfe-masters-project/src/models/base
Scalers directory: /home/alfred/projects/mscfe-masters-project/src/models/scaler
Best models directory: /home/alfred/projects/mscfe-masters-project/src/models/best



# Read raw data

In [4]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [5]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
52878,2024-12-13 13:10:00,72.89000,72.9200,72.87000,72.92000,CAC40_ETF
252929,2024-12-03 00:35:00,95464.45000,95464.4600,95356.01000,95404.00000,BTCUSDT
226303,2024-11-23 18:40:00,0.21530,0.2155,0.21530,0.21550,TRXUSDT
182588,2024-11-08 14:25:00,596.26000,596.7300,595.89000,596.58000,BNBUSDT
175410,2024-11-06 02:35:00,0.20005,0.2010,0.19928,0.20023,DOGEUSDT


## Placeholders

In [6]:
dict_data = {}
dict_cv_features = {}

# Features Engineering

## Create and normalizing features

In [7]:
# Initializing the FeaturesEngineering obj
feat = FeaturesEngineering()

# Default config
feat.default_config

{'ema': [8, 21, 55],
 'macd': {'fast': 12, 'slow': 26, 'signal': 9},
 'rsi': [14],
 'bb': {'timeperiod': 20, 'nbdevup': 2, 'nbdevdn': 2},
 'atr': {'timeperiod': 14},
 'stoch': {'fastk_period': 14, 'slowk_period': 3, 'slowd_period': 3},
 'cci': {'timeperiod': 20},
 'willr': {'timeperiod': 14}}

1. Create features from pairs data
2. Perform normalization / standardization
3. Save scaler for future use like testing and strategy

In [8]:
scaler_type = "StandardScaler"
dict_data['df_feat'] = {}
dict_data['df_norm'] = {}
dict_data['scaler'] = {}
i = 1

for p in pairs:

    print(f"({i}) Pairs: {p}")
    tickerX, tickerY = p.split(' ')
    
    # 1. Create features from pairs data
    df_feat = feat.create_features(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        config=None, # using default config in the FeaturesEngineering
        dropna=True
    )

    # 2. Perform normalization / standardization
    df_norm, scaler = feat.normalize_features(df_feat, scaler_type)
    df_norm = df_norm.rename(columns={'Spread':'NormalizedSpread'})
    
    # 3. Save scaler for future use like testing and strategy
    scaler_name = f"{scaler_type.lower()}_{p.replace(' ', '_')}.pkl"
    dir_scaler = os.path.join(dir_models_scaler, scaler_name)
    pypickle.save(dir_scaler, scaler, overwrite=pypickle_overwrite)

    dict_data['df_feat'][p] = df_feat
    dict_data['df_norm'][p] = df_norm
    dict_data['scaler'][p] = scaler

    i+=1
    print('')
    

(1) Pairs: META DAXEX_ETF
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_ETHUSDT.pkl]

(6) Pairs: EQQQ_ETF BNBUSDT
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
[pypickle] Pickle file saved: [/home/alfr

## Perform Lasso cross validation

In [9]:
models = Models(use_xgboost_gpu=True)

In [10]:
cv=5
random_seed=42
max_iter=10000
tol=1e-4
i = 1

# for p in pairs.iloc[0:2]:
for p in pairs:

    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    feature_cols = df_norm.columns.tolist()[:-1]
    target_col = df_norm.columns.tolist()[-1]

    # 2. Run Lasso Cross Validation
    df_X = df_norm[feature_cols]
    lasso = models.lasso_cross_validation(df_norm, feature_cols, target_col, cv, random_seed, max_iter, tol)
    selected_features = df_X.columns[abs(lasso.coef_) > 0].tolist()

    # 3. Prepare selected features determined by LassoCV
    dict_cv_features[p] = selected_features
    print(f"LassoCV features: {selected_features}")

    i+=1
    print('')

# 4. Save all LassoCV features dictionary
lasso_cv_name = f"lasso_cv_features.pkl"
dir_lasso = os.path.join(dir_models, lasso_cv_name)
pypickle.save(dir_lasso, dict_cv_features, overwrite=pypickle_overwrite)

(1) Pairs: META DAXEX_ETF
Best Alpha: 0.001498163835994244
Non-zero Coefficients: 14
LassoCV features: ['META_EMA8', 'META_EMA55', 'META_RSI14', 'META_ATR14', 'META_StochD3', 'META_CCI20', 'META_WILLR14', 'DAXEX_ETF_EMA8', 'DAXEX_ETF_EMA21', 'DAXEX_ETF_MACD', 'DAXEX_ETF_BBupper', 'DAXEX_ETF_BBmiddle', 'DAXEX_ETF_BBlower', 'DAXEX_ETF_StochD3']

(2) Pairs: AMZN BNBUSDT
Best Alpha: 0.0009823230359649105
Non-zero Coefficients: 15
LassoCV features: ['AMZN_EMA8', 'AMZN_EMA21', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_WILLR14', 'BNBUSDT_EMA8', 'BNBUSDT_EMA55', 'BNBUSDT_MACD', 'BNBUSDT_RSI14', 'BNBUSDT_ATR14', 'BNBUSDT_StochD3', 'BNBUSDT_CCI20', 'BNBUSDT_WILLR14']

(3) Pairs: AMZN TRXUSDT
Best Alpha: 0.0009993215252578579
Non-zero Coefficients: 8
LassoCV features: ['AMZN_EMA8', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_CCI20', 'AMZN_WILLR14', 'TRXUSDT_MACD']

(4) Pairs: SPY_ETF AVAXUSDT
Best Alpha: 0.0005239636240581893
Non-zero Coefficients: 20
Las

True

# Hyperparameters Tunning

## Ridge Regression

In [12]:
dir_best = os.path.join(dir_models_best, 'ridge')
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag'],
}

verbose = False
tuned_params = []
i = 1

# for p in pairs.iloc[0:1]:
for p in pairs:

    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)

    # 3. Run Hyperparameters tunning
    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=df_lasso, p=p, model_func=models.ridge_regression, param_grid=param_grid, verbose=verbose)

    stats = {
        'pair':p, 
        'best_mse': best_mse,
        'data_points': len(df_lasso),
        'features': len(df_lasso.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

    print(f"Best MSE: {best_mse:.6f}")
    print(f"alpha: {best_params['alpha']}, solver: {best_params['solver']}")

    # 4. Save best model future use like testing and strategy
    model_name = f"best_ridge_{p.replace(' ', '_')}.pkl"
    dir_model_name = os.path.join(dir_best, model_name)
    pypickle.save(dir_model_name, best_model, overwrite=pypickle_overwrite)

    i+=1
    print('')

dir_param = os.path.join(dir_best, 'best_model_results.csv')
df_ridge = pd.DataFrame(tuned_params).set_index('pair')
df_ridge.to_csv(dir_param)
display(df_ridge)

(1) Pairs: META DAXEX_ETF
Best MSE: 0.002754
alpha: 1, solver: sparse_cg
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/ridge/best_ridge_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
Best MSE: 0.001153
alpha: 0.001, solver: lsqr
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/ridge/best_ridge_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
Best MSE: 0.000376
alpha: 1, solver: lsqr
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/ridge/best_ridge_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
Best MSE: 0.007090
alpha: 0.01, solver: sparse_cg
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/ridge/best_ridge_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
Best MSE: 0.000871
alpha: 0.001, solver: svd
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/ridge/best_ridge_SPY_ETF_ETHUSDT.pkl

,best_mse,data_points,features,alpha,solver
pair,,,,,
META DAXEX_ETF,0.002754,1145,15,1.000,sparse_cg
AMZN BNBUSDT,0.001153,4991,16,0.001,lsqr
AMZN TRXUSDT,0.000376,4991,9,1.000,lsqr
SPY_ETF AVAXUSDT,0.007090,4997,21,0.010,sparse_cg
SPY_ETF ETHUSDT,0.000871,4997,16,0.001,svd
EQQQ_ETF BNBUSDT,0.000715,3217,13,0.001,sparse_cg
BNBUSDT ADAUSDT,0.000150,31051,9,0.010,sag
BNBUSDT BTCUSDT,0.000093,31051,9,0.001,svd
BNBUSDT TRXUSDT,0.000148,31051,12,0.100,sag


## Xgboost Regression

In [14]:
dir_best = os.path.join(dir_models_best, 'xgb')
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],  # Step size shrinkage
    'n_estimators': [50, 100, 200],  # Number of boosting rounds
    'max_depth': [5, 7, 9],  # Maximum tree depth
}

verbose = False
tuned_params = []
i = 1

# for p in pairs.iloc[0:2]:
for p in pairs:

    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)

    # 3. Run Hyperparameters tunning
    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=df_lasso, p=p, model_func=models.xgboost_regression, param_grid=param_grid, verbose=verbose)

    stats = {
        'pair':p, 
        'best_mse': best_mse,
        'data_points': len(df_lasso),
        'features': len(df_lasso.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

    print(f"Best MSE: {best_mse:.6f}")
    print(f"learning_rate: {best_params['learning_rate']}, n_estimators: {best_params['n_estimators']}, max_depth: {best_params['max_depth']}")

    # 4. Save best model future use like testing and strategy
    model_name = f"best_xgb_{p.replace(' ', '_')}.pkl"
    dir_model_name = os.path.join(dir_best, model_name)
    pypickle.save(dir_model_name, best_model, overwrite=pypickle_overwrite)

    i+=1
    print('')

dir_param = os.path.join(dir_best, 'best_model_results.csv')
df_xgb = pd.DataFrame(tuned_params).set_index('pair')
df_xgb.to_csv(dir_param)
display(df_xgb)

(1) Pairs: META DAXEX_ETF
Best MSE: 0.026589
learning_rate: 0.05, n_estimators: 50, max_depth: 7
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/xgb/best_xgb_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
Best MSE: 0.006577
learning_rate: 0.1, n_estimators: 200, max_depth: 5
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/xgb/best_xgb_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
Best MSE: 0.000608
learning_rate: 0.05, n_estimators: 200, max_depth: 5
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/xgb/best_xgb_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
Best MSE: 0.323226
learning_rate: 0.05, n_estimators: 200, max_depth: 7
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/best/xgb/best_xgb_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
Best MSE: 0.003511
learning_rate: 0.1, n_estimators: 200, max_depth: 5
[pypickle] Pickl

,best_mse,data_points,features,learning_rate,n_estimators,max_depth
pair,,,,,,
META DAXEX_ETF,0.026589,1145,15,0.05,50,7
AMZN BNBUSDT,0.006577,4991,16,0.10,200,5
AMZN TRXUSDT,0.000608,4991,9,0.05,200,5
SPY_ETF AVAXUSDT,0.323226,4997,21,0.05,200,7
SPY_ETF ETHUSDT,0.003511,4997,16,0.10,200,5
EQQQ_ETF BNBUSDT,0.028995,3217,13,0.10,200,5
BNBUSDT ADAUSDT,0.000159,31051,9,0.10,200,5
BNBUSDT BTCUSDT,0.000149,31051,9,0.10,200,9
BNBUSDT TRXUSDT,0.000135,31051,12,0.10,200,5
